In [ ]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.3 MB/s 
     |████████████████████████████████| 1.6 MB 37.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization

from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Embedding, Add

from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPooling1D
from keras.models import Sequential, Model
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Read training dataset

In [ ]:
#read train dataset
train=  pd.read_csv('/content/drive/MyDrive/Myocardial Infarction Detection/mitbih_train.csv',encoding='utf-8')
test= pd.read_csv('/content/drive/MyDrive/Myocardial Infarction Detection/mitbih_test.csv',encoding='utf-8')

## convert label to categorical

In [ ]:
from keras.utils.np_utils import to_categorical
X, y = train.iloc[: , :-1], train.iloc[: , -1]
X_train, X_valid, y_train, y_valid= train_test_split(X,y,test_size=0.2)
X_test, y_test = test.iloc[: , :-1], test.iloc[: , -1]
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)
y_test=to_categorical(y_test)

In [ ]:
print("X shape=" +str(X_train.shape))
print("y shape=" +str(y_train.shape))

print("testX shape=" +str(X_test.shape))
print("testy shape=" +str(y_test.shape))

## Define some of function that are used as matixs 

In [ ]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Optimize and build mode 

In [ ]:
def build_model(hp):
  

    dropout_min  =  .1
    dropout_max  =  0.9
    dropout_step =  0.1
    
    model = keras.Sequential()
    
    model.add(
        Conv1D(
            filters=hp.Choice(
                'num_filters1',[128,256] ),input_shape=(187,1), activation='relu', kernel_size=hp.Choice('kernel_size1', [4,5])))
 

    model.add(MaxPooling1D(pool_size=hp.Choice('Pool_Size1', [2])))

    model.add(Dropout(rate=hp.Float('dropout1', min_value=dropout_min, max_value=dropout_max , default=dropout_step)))

    model.add(
        Conv1D(
            filters=hp.Choice(
                'num_filters2',[128,256] ),
            activation='relu',
            kernel_size=hp.Choice('kernel_size2', [4,5]),
           ))
  
    model.add(MaxPooling1D(pool_size=hp.Choice('Pool_Size2', [2])))

    model.add(Dropout(rate=hp.Float('dropout2', min_value=dropout_min, max_value=dropout_max , default=dropout_step)))

    model.add(Flatten())
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=20,
                                                 max_value=500,
                                                 step=20),activation = 'relu', 
                          ))
    

    model.add(layers.Dense(units=hp.Int('unit2',  min_value=20,
                                                 max_value=400,
                                                 step=20),activation = 'relu'))

    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam( hp.Float('learning_rate',min_value=1e-4,max_value=1e-2)),loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m] )
    return model



In [ ]:

tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=1,
    project_name='/content/drive/MyDrive/Myocardial Infarction Detection/CNN_Layer2')


In [ ]:
epoch=20
BATCH_SIZE=1000
early_stopping = EarlyStopping(monitor='val_acc', patience=30, verbose=1)
callback_list = [ early_stopping ]

# split training data into stratified train/dev sets

h=tuner.search(X_train, y_train,
             epochs=epoch,
             batch_size=BATCH_SIZE, 
             callbacks=callback_list, validation_data=((X_test, y_test)) )
            
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.save("/content/drive/MyDrive/Myocardial Infarction Detection/model/CNN_Layer2.h5")

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
history=model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=epoch, validation_data=(X_test, y_test))

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Evaluating model

In [ ]:
AccuracyTest=[]
PrecisionTest=[]
RecallTest=[]
F1Test=[]
y_true=[]
for element in y_test:
  y_true.append(np.argmax(element))
for i in range(0,1): 
    prediction_proba=model.predict(X_test)
    y_pred=np.argmax(prediction_proba,axis=1)

    Accurcy_Test= accuracy_score(y_true,y_pred)
    Precision_Test=precision_score(y_true, y_pred,average='micro')

    Recall_Test=recall_score(y_true, y_pred, average='micro')
    F1_Test=f1_score(y_true, y_pred, average='micro') 

    AccuracyTest.append(round(100*Accurcy_Test, 2))
    PrecisionTest.append(round(100*Precision_Test, 2))
    RecallTest.append(round(100*Recall_Test, 2))
    F1Test.append(round(100*F1_Test, 2))

In [ ]:
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),
             'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)

In [ ]:
print(classification_report(y_true, y_pred, target_names=['Normal','Artial Premature','Premature ventricular contraction','Fusion of ventricular and normal','Fusion of paced and normal']))